In [ ]:
import urllib.request
import html5lib
import requests
import regex as re
import bs4
import random
from bs4 import BeautifulSoup
import pandas as pd
import time

data = pd.read_csv('urls/urls.csv')

df = pd.DataFrame(columns = ['Url Index', 'Url', 'Title', 'Sub Title', 'Author', 'Author Url', 'Reading Time', 'Clap Count', 'Image Count/Sources', 'Article Text'])

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.42'}

scrapped_links = []

for i in range(data.shape[0]):
    
        cur_link_data = []
        url_to_scrap = data.iloc[i]['url']
        
        try:
        
            r = requests.get(url_to_scrap, headers=headers, timeout = 20)
            r_text = r.text
            soup = BeautifulSoup(r_text, "html5lib")

            try:
                title = soup.find('h1').contents[0].text
            except AttributeError:
                title_raw = (soup.find('title')).text
                title = title_raw[:title_raw.find("|")]

            if not title:
                continue

            try:
                if "Written by" in soup.find('h2').contents[0].text:
                    sub_title = "None"
                else:
                    sub_title = soup.find('h2').contents[0].text
            except AttributeError:
                sub_title = "None"

            try:
                author_name_raw = (soup.find("h2", {"class": "pw-author-name"})).text
                author_name = author_name_raw[11:]
            except AttributeError:
                author_name = "None"

            try:
                author_url_raw = (soup.find("h2", {"class": "pw-author-name"})).parent['href']
                author_url = "https://medium.com" + author_url_raw[:author_url_raw.find("?")]
            except AttributeError:
                author_url = "None"

            r_content = r.content.decode("utf-8")
            reading_time = r_content[r_content.find(" min read")-2:r_content.find(" min read")]
            if reading_time[0] == '"':
                reading_time = reading_time[1:]

            try:
                clap_count = r_content.split("clapCount\":")[1]
            except IndexError:
                clap_count = r_content.split("clapCount\":")[0]
            clap_count = clap_count[0:clap_count.find(",")]

            reg = re.compile('(https:\/\/miro\.medium\.com\/v2\/resize:fit:)[\d]+(\/)(format:webp\/|)(\w|\d|-|\*)+(\.jpeg|\.png|\.gif)')
            img_tags = soup.find_all('picture')
            img_srcs = str(len(img_tags))
            for img in img_tags:
                mos = reg.search(str(img))
                try:
                    img_srcs += ", " + str(mos[0])
                except TypeError:
                    continue

            text = ""
            divTag = soup.find_all("div", {"class": "ch bg dx dy dz ea"})
            for tag in divTag:
                tdTags = tag.find_all("p")
                for tag in tdTags:
                    text += tag.text

            if not text:
                continue

            df.loc[i] = [i, url_to_scrap, title, sub_title, author_name, author_url, reading_time, clap_count,img_srcs, text]
            scrapped_links.append(url_to_scrap)
            print(i)
            time.sleep(1)
        
        except:
            df.to_csv("data.csv", encoding='utf-8')
            time.sleep(5)

df.to_csv("data.csv", encoding='utf-8')
print("-----Finished------")

In [ ]:
#------------Individual component scraping codes just for demonstration----------#
print(soup.find('h1').contents[0].text) #Extract title
print(soup.find('h2').contents[0].text) #Extract sub title
print((soup.find("h2", {"class": "pw-author-name"})).text) #Extract author name
print((soup.find("h2", {"class": "pw-author-name"})).parent['href']) #Extract author url
print((soup.find("div", {"class": "speechify-ignore"})).text) #Extract reading time
#reading_time_raw = (soup.find("div", {"class": "speechify-ignore"})).text
#reg = re.compile('(\d)+(?= min read)')
#mo = reg.search(reading_time_raw)
#reading_time = mo[0]
divTag = soup.find_all("div", {"class": "ch bg dx dy dz ea"}) #Extract article text
#for tag in divTag:
    #tdTags = tag.find_all("p")
    #for tag in tdTags:
        #print(tag.text)
reg = re.compile('(https:\/\/miro\.medium\.com\/v2\/resize:fit:)[\d]+(\/)(\w|\d|-|\*)+(\.jpeg|\.png|\.gif)') #Extract Images
#reg = re.compile('(https:\/\/miro\.medium\.com\/v2\/resize:fit:)[\d]+(\/)(format:webp\/|)(\w|\d|-|\*)+(\.jpeg|\.png|\.gif)')
#img_tags = soup.find_all('picture')
#for img in img_tags:
    #mos = reg.search(str(img))
    #print(mos[0])